In [19]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [20]:
import galois
import numpy as np
import threading
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

np.set_printoptions(threshold=np.inf)

In [21]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [22]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [23]:
#provo co due matrici diagonali NON funziona
valori_diagonale = np.arange(1,127)
A = np.diag(valori_diagonale)

perm = np.random.permutation(126) #permutazione degli indici casuale
A_primo = A[perm, :]
A_primo = A_primo[:, perm]

A = field(A)
A_primo = field(A_primo)

In [24]:
A = field.Random((k, n_k))
#print(A)

In [25]:
id_mat = np.eye(126, dtype=int)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin1 = id_mat.copy()
matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin1 = field(matrice_bin1)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin2 = id_mat.copy()
matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin2 = field(matrice_bin2)

print(matrice_bin1)
print(matrice_bin2)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [26]:
#np.dot prodotto di due  array
A_primo_intermedio = field(np.dot(matrice_bin1, A))
#print(A_primo_intermedio)

In [27]:
A_primo = matrice_bin1 @ A @matrice_bin2
A_primo = field(A_primo)
#print(A_primo)

In [28]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [field(0)] * A_primo.shape[0]

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    somma = field(np.sum([riga])%127)
    return somma
    #print("Riga : ", riga) #questi vanno eliminati poi
    #print("Tipo riga: ", type(riga)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print("Somma: ", somma) #questi vanno eliminati poi
    #print("Tipo somma: ",type(somma)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print(somme_rig)

#vado ad estrarre il primo ed il secondo elemento di ogni colonna
def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    return colonna[:4]  # Restituisce i primi quattro elementi della colonna



#eseguo in parallelo somma_riga
def esegui_somma_righe_parallelo(matrice):
    somme_rig = [0] * matrice.shape[0]  # Inizializza una lista per le somme

    with ThreadPoolExecutor() as executor:
        executor.map(somma_riga, [matrice]*matrice.shape[0], range(matrice.shape[0]))
    return somme_rig

#eseguo in parallelo estrazione
def esegui_estrazione_seriale(matrice):
    for indice_colonna in range(matrice.shape[1]):
        estrai_da_colonna(matrice, indice_colonna)

# Funzione per applicare una permutazione ciclica a un insieme di colonne
def permutazione_ciclica(matrice, blocco_dim):
    matrice_permutata = matrice.copy()
    
    # Eseguiamo una permutazione ciclica su tutte le colonne a gruppi di 'blocco_dim'
    for i in range(0, matrice.shape[1], blocco_dim):
        fine_blocco = min(i + blocco_dim, matrice.shape[1])
        # Cicla il blocco selezionato
        matrice_permutata[:, i:fine_blocco] = np.roll(matrice[:, i:fine_blocco], shift=1, axis=1)
    
    return field(matrice_permutata)


In [33]:
#fino a size 3 provato e funziona
while True:
    id_mat = np.eye(126, dtype=int)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin1 = id_mat.copy()
    matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin1 = field(matrice_bin1)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin2 = id_mat.copy()
    matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin2 = field(matrice_bin2)

    #print(matrice_bin1)
    #print(matrice_bin2)
    A_primo = matrice_bin1 @ A @ matrice_bin2
    A_primo = field(A_primo)
    #print(A_primo)"""
    somme_rig = [field(0)] * A_primo.shape[0]

    #lista per memorizzare i primi elementi della colonna
    primi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i secondi elemnti della colonna
    secondi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i terzi elementi della colonna
    terzi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i quarti elemnti della colonna
    quarti_colonne = [0] * A_primo.shape[1]

    #QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
    #PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

    colonne_finali = []
    esegui_somma_righe_parallelo(A_primo)
    #ordino in maniera crescente per valore gli indici
    indici_ordinati = np.argsort(somme_rig)
        #print(indici_ordinati)

        #applico gli swap
    A_canonica_righe = A_primo[indici_ordinati, :]
    #ora si passa alle colonne
    esegui_estrazione_seriale(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    #qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
    A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

    #print(A_CF)

    #devo fare lo stesso per la matrice A per dimostrare che sono CF

    colonne_finali = []

    #creo un thread per ogni colonna
    esegui_somma_righe_parallelo(A)

    #print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

    #ordino in maniera crescente i valori
    indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
    A_canonica_righe = A[indici_ordinati, :]

    #print(A_canonica_righe)

    #ora si passa alle colonne
    esegui_estrazione_seriale(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
    #print(A_CF1)

    zero = np.subtract(A_CF1,A_CF)
    zero = np.count_nonzero(zero)
    print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")
    if zero == 0:
        break


Le matrici differiscono per  618  elementi su 15876 (126*126)
Le matrici differiscono per  494  elementi su 15876 (126*126)
Le matrici differiscono per  374  elementi su 15876 (126*126)
Le matrici differiscono per  743  elementi su 15876 (126*126)
Le matrici differiscono per  0  elementi su 15876 (126*126)


In [34]:
print("Questa è A:", A)
print("\nQuesta è A': ", A_primo)
print("\nQuesta è la forma canonica: ", A_CF)

Questa è A: [[  1  20  99  76  42  68  12   0   4  17  51 112  85  11 113  25  74  19
   75  65  72 116  56  99  23  73  72  70  98   1  76  44  61  92  58  54
   25  90  34  82   7  59  85  86  50  54  82 103 118 117 114   3  76  63
   45  74  99  32 112   2  85   0  82  37  40 109  20   4  27 111 112 120
   57 104  47  72 104  56 123  45 119 112 114   4  44 120  19 111 125   7
   67  21  11  15  85  29 111  42   0   3  38 126 107   1  63  29  47  16
    1  39  19  51   2 105  10  53  74  63  96  98  15   5 114  75  22  22]
 [ 44  35  39   5 106  55  99  17  76 125  98 123 111  20  87  76 113  41
   46  25  13  64 111  20  69  30  83  23 113  74  82  36  69  12  96 124
    6 120  38  60  82 103  63  94  86  28  67   1  58 103  14  67  73  26
   28   3  77 117   7  85  93  22   7  43   4  16  89  30  54  62   6 119
   48  57  48  46   7  82  69  30  35   4  49   6 125   5  28  21  58  78
   47  18  11  94  63  67 101  89  48  52 120  97  27  27 112  30  57  33
   81  38 123  73  94  40

[[  1  20  99  76  42  68  12   0   4  17  51 112  85  11 113  25  74  19
   75  65  72 116  56  99  23  73  72  70  98   1  76  44  61  92  58  54
   25  90  34  82   7  59  85  86  50  54  82 103 118 117 114   3  76  63
   45  74  99  32 112   2  85   0  82  37  40 109  20   4  27 111 112 120
   57 104  47  72 104  56 123  45 119 112 114   4  44 120  19 111 125   7
   67  21  11  15  85  29 111  42   0   3  38 126 107   1  63  29  47  16
    1  39  19  51   2 105  10  53  74  63  96  98  15   5 114  75  22  22]
 [ 44  35  39   5 106  55  99  17  76 125  98 123 111  20  87  76 113  41
   46  25  13  64 111  20  69  30  83  23 113  74  82  36  69  12  96 124
    6 120  38  60  82 103  63  94  86  28  67   1  58 103  14  67  73  26
   28   3  77 117   7  85  93  22   7  43   4  16  89  30  54  62   6 119
   48  57  48  46   7  82  69  30  35   4  49   6 125   5  28  21  58  78
   47  18  11  94  63  67 101  89  48  52 120  97  27  27 112  30  57  33
   81  38 123  73  94  40  87  51  83

In [35]:
import time

In [37]:
#Voglio vedere il tempo di esecuzione avendo già le matrici

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

colonne_finali = []
start_time = time.perf_counter()

esegui_somma_righe_parallelo(A_primo)
#ordino in maniera crescente per valore gli indici
indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
A_canonica_righe = A_primo[indici_ordinati, :]
#ora si passa alle colonne
esegui_estrazione_seriale(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

#qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

#print(A_CF)

#devo fare lo stesso per la matrice A per dimostrare che sono CF

colonne_finali = []

#creo un thread per ogni colonna
esegui_somma_righe_parallelo(A)

#print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

#ordino in maniera crescente i valori
indici_ordinati = np.argsort(somme_rig)
#print(indici_ordinati)

#applico gli swap
A_canonica_righe = A[indici_ordinati, :]

#print(A_canonica_righe)

#ora si passa alle colonne
esegui_estrazione_seriale(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)

for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
#print(A_CF1)
end_timer = time.perf_counter()
durata = (end_timer - start_time)*1000
print("Durata esecuzione: ", durata, " ms")

zero = np.subtract(A_CF1,A_CF)
zero = np.count_nonzero(zero)
print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")

Durata esecuzione:  283.153702999698  ms
Le matrici differiscono per  0  elementi su 15876 (126*126)
